In [ ]:
# Perform git clone as follows

!git clone TODO
!cp -r cse579_HW2/* .

# !NOTE!: Once you are done, copy your implementation of policy gradient, actor critic and
# in the notebook here back to the python script
# when submiting your code

!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install setuptools wheel
!pip install gym==0.26.2
!pip install gymnasium
!pip install gymnasium-robotics[mujoco-py]
!pip install gym-notices==0.0.8
!pip install matplotlib
!pip install mujoco
!pip install free-mujoco-py
!pip install pybullet
!pip install tqdm
!pip install imageio[ffmpeg] matplotlib
import os
os.environ['LD_PRELOAD']=':/usr/lib/x86_64-linux-gnu/libGLEW.so'



In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from torch import optim
import gymnasium as gym
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from utils import log_density, ReplayBuffer
from networks import PGPolicy, PGBaseline
import random
from rollouts import evaluate, evaluate_agent, rollout, rollout_frames
import sac
from agents import GenericACAgent, train_agent
import imageio
from IPython.display import Video

In [2]:
print("sdd")

sdd


In [ ]:
def train_model(policy, baseline, trajs, policy_optim, baseline_optim, device, gamma=0.99, baseline_train_batch_size=64,
                baseline_num_epochs=5):
    states_all = []
    actions_all = []
    returns_all = []
    for traj in trajs:
        states_singletraj = traj['observations']
        actions_singletraj = traj['actions']
        rewards_singletraj = traj['rewards']
        returns_singletraj = np.zeros_like(rewards_singletraj)
        running_returns = 0
        for t in reversed(range(0, len(rewards_singletraj))):
            running_returns = rewards_singletraj[t] + gamma * running_returns
            returns_singletraj[t] = running_returns
        states_all.append(states_singletraj)
        actions_all.append(actions_singletraj)
        returns_all.append(returns_singletraj)
    states = np.concatenate(states_all)
    actions = np.concatenate(actions_all)
    returns = np.concatenate(returns_all)

    # Normalize the returns
    returns = (returns - returns.mean()) / returns.std() + 1e-8

    criterion = torch.nn.MSELoss()
    n = len(states)
    arr = np.arange(n)
    for epoch in range(baseline_num_epochs):
        np.random.shuffle(arr)
        for i in range(n // baseline_train_batch_size):
            batch_index = arr[baseline_train_batch_size * i: baseline_train_batch_size * (i + 1)]
            batch_index = torch.LongTensor(batch_index).to(device)
            inputs = torch.Tensor(states).to(device)[batch_index]
            target = torch.Tensor(returns).to(device)[batch_index]
            
            #========== TODO: start ==========
            # Train baseline by regressing onto returns.
            # Hint: Regress the baseline from each state onto the above
            # computed return to go. You can use similar code to behavior cloning to do so. This should be
            # 2 lines of code



            #========== TODO: END ==========
            baseline_optim.zero_grad()
            loss.backward()
            baseline_optim.step()

    action, std, logstd = policy(torch.Tensor(states).to(device))
    log_policy = log_density(torch.Tensor(actions).to(device), policy.mu, std, logstd)
    baseline_pred = baseline(torch.from_numpy(states).float().to(device))
    #========== TODO: start ==========
    # Train policy by optimizing surrogate objective: -log prob * (return - baseline)
    # Hint: Policy gradient is given by: \grad log prob(a|s)* (return - baseline)
    # Hint: Return is computed above, you can computer log_probs using the log_density function imported.
    # Hint: You can predict what the baseline outputs for every state.
    # Hint: Then simply compute the surrogate objective by taking the objective as -log prob * (return - baseline)
    # Hint: You can then use standard pytorch machinery to take *one* gradient step on the policy


    
    #========== TODO: END ==========
    policy_optim.zero_grad()
    loss.backward()
    policy_optim.step()

    del states, actions, returns, states_all, actions_all, returns_all


# Training loop for policy gradient
def simulate_policy_pg(env, policy, baseline, num_epochs=200, batch_size=100,
                       gamma=0.99, baseline_train_batch_size=64, baseline_num_epochs=5, print_freq=10, device = "cuda", render=False):
    policy_optim = optim.Adam(policy.parameters())
    baseline_optim = optim.Adam(baseline.parameters())

    for iter_num in range(num_epochs):
        sample_trajs = []

        # Sampling trajectories
        for it in range(batch_size):
            sample_traj = rollout(
                env,
                policy,
                render=False)
            sample_trajs.append(sample_traj)

        # Logging returns occasionally
        if iter_num % print_freq == 0:
            rewards_np = np.mean(np.asarray([traj['rewards'].sum() for traj in sample_trajs]))
            path_length = np.max(np.asarray([traj['rewards'].shape[0] for traj in sample_trajs]))
            print("Episode: {}, reward: {}, max path length: {}".format(iter_num, rewards_np, path_length))

        # Training model
        train_model(policy, baseline, sample_trajs, policy_optim, baseline_optim, device, gamma=gamma,
                    baseline_train_batch_size=baseline_train_batch_size, baseline_num_epochs=baseline_num_epochs)

In [ ]:
class ActorCriticAgent(GenericACAgent):
    def update_actor(self, obs):
        dist = self.actor(obs)
        # The .rsample() is used to sample using the reparameterization trick to allow for backpropagation
        action = dist.rsample()
        log_prob = dist.log_prob(action).sum(-1, keepdim=True)
        #========== TODO: start ==========
        # Implement the actor update
        # Compute the Q values of the action using self.critic(obs, action). In this case it is a single instead of
        # double Q function so you do not need to take a minimum.
        # The policy loss is the mean over the negative Q values i.e we want to maximize the Q values

        
        
        #========== TODO: end ==========
        # optimize the actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        return actor_loss.item(), 0, 0

    def update_critic(self, obs, action, reward, next_obs, not_done_no_max):
        #========== TODO: start ==========
        # Train the single Q function:
        # Hint: Step 1: Compute current Q predictions using the obs and action and self.critic()
        # Hint: Step 2: Compute q targets using reward + critic_target * not_done_no_max for next_obs and
        # next_actions actions sampled from the current policy. Use torch.no_grad() for this step to disable
        # gradient flow to the critic_target and the actor.
        # Hint: Step 3: Compute Bellman error as mean squared error between q_predictions and q_targets
       


        #========== TODO: end ==========
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        return critic_loss.item()


In [ ]:
class SACAgent(GenericACAgent):
    def update_actor(self, obs):        
        #========== TODO: start ==========
        # Sample actions and the log_prob of the actions from the actor given obs. Hint: This is the same as AC agent.
        # Get the two Q values from the double Q function critic and take the minimum value. Then calculate the actor loss which
        # is defined by self.alpha * log_prob - actor_Q. Make sure that gradient does not flow through the alpha paramater. 

        
        #========== TODO: end ==========
        # optimize the actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update the temperature parameter toachieve entropy close to the target entropy
        self.log_alpha_optimizer.zero_grad()
        alpha_loss = (self.alpha *
                      (-log_prob - self.target_entropy).detach())
        alpha_loss = alpha_loss.mean()
        alpha_loss.backward()
        self.log_alpha_optimizer.step()
        return actor_loss.item(), -log_prob.mean().item(), alpha_loss.item()
    
    def update_critic(self, obs, action, reward, next_obs, not_done_no_max):
        #========== TODO: start ==========
        # Train the double Q function:
        # Hint step 1: Sample the next_action and log_prob of the next action using the self.actor and the next_obs. Use the code 
        # below in update_actor as a reference on how to do this
        
        # Hint step 2: Sample the two target Q values from the critic_target using next_obs and the sampled next_action. 
        # Take these numbers, and get the target value by taking the min of the 2 q values and then subtracting self.alpha*log_prob
        # The target Q is the reward + (not_done_no_max * discount * target_value)
        
        # Hint step 3:
        # Sample the current Q1 and Q2 values of the current state using the critic. The loss is mse(Q1, targetQ) + mse(Q2 + target Q)

        
        #========== TODO: end ==========
        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        return critic_loss.item()

In [3]:
class Args:
    def __init__(self, task, env, test=False, render=False):
        self.task = task # 'pg' or 'actor_critic' or 'sac'
        self.env = env # pendulum or ant
        self.test = test # whether test only
        self.render = render # whether to render during test
args = Args('pg', "pendulum", False,  False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
if __name__ == '__main__':
    if args.env == 'pendulum':
        env = gym.make("InvertedPendulum-v5", render_mode='human' if args.render else None)
        max_episode_steps = 200
        env = gym.wrappers.TimeLimit(env, max_episode_steps=max_episode_steps)
    elif args.env == 'ant':
        max_episode_steps = 500
        env = gym.make("Ant-v5", render_mode='human' if args.render else None)
        env = gym.wrappers.TimeLimit(env, max_episode_steps=max_episode_steps)
    else:
        raise ValueError('Invalid environment')

    obs_size = env.observation_space.shape[0]
    ac_size = env.action_space.shape[0]
    action_range = [
        float(env.action_space.low.min()),
        float(env.action_space.high.max())]

    if args.task == 'pg':
        # Define policy and value function
        hidden_dim_pol = 64
        hidden_depth_pol = 2
        hidden_dim_baseline = 64
        hidden_depth_baseline = 2
        policy = PGPolicy(obs_size, ac_size, hidden_dim=hidden_dim_pol,
                          hidden_depth=hidden_depth_pol)
        baseline = PGBaseline(obs_size, hidden_dim=hidden_dim_baseline,
                              hidden_depth=hidden_depth_baseline)
        policy.to(device)
        baseline.to(device)

        # Training hyperparameters
        num_epochs = 150
        batch_size = 64
        gamma = 0.99
        baseline_train_batch_size = 64
        baseline_num_epochs = 5
        print_freq = 10

        if not args.test:
            # Train policy gradient
            simulate_policy_pg(env, policy, baseline, num_epochs=num_epochs, batch_size=batch_size,
                               gamma=gamma, baseline_train_batch_size=baseline_train_batch_size, device=device,
                               baseline_num_epochs=baseline_num_epochs, print_freq=print_freq, render=args.render)
            torch.save(policy.state_dict(), 'pg_final.pth')
        else:
            print('loading pretrained pg')
            policy.load_state_dict(torch.load(f'pg_final.pth'))
        evaluate(env, policy, num_validation_runs=100, render=args.render)
    else:
        num_train_steps = 100_000
        num_seed_steps = 5000
        eval_frequency = 10_000
        num_eval_episodes = 10
        replay_buffer = ReplayBuffer(obs_size, ac_size, num_train_steps, device)
        hidden_dim = 256
        hidden_depth = 2
        batch_size = 256
        discount_factor = 0.99
        if args.task == 'actor_critic':
            agent = ActorCriticAgent(
                obs_dim=obs_size,
                action_dim=ac_size,
                action_range=action_range,
                device=device,
                discount=discount_factor,
                actor_lr=3e-4,
                critic_lr=3e-4,
                critic_tau=5e-3,
                batch_size=batch_size,
                hidden_dim=hidden_dim,
                hidden_depth=hidden_depth,
                double_critic=False,
                temperature=False
            )

        elif args.task == "sac":
            agent = sac.SACAgent(
                obs_dim=obs_size,
                action_dim=ac_size,
                action_range=action_range,
                device=device,
                discount=discount_factor,
                init_temperature=0.1,
                alpha_lr=3e-4,
                actor_lr=3e-4,
                critic_lr=3e-4,
                critic_tau=0.005,
                batch_size=batch_size,
                target_entropy=-ac_size,
                hidden_dim=hidden_dim,
                hidden_depth=hidden_depth,
                double_critic=True,
                temperature=True
            )
        else:
            raise ValueError('Invalid task')

        if not args.test:
            train_agent(agent,
                        env,
                        num_train_steps=num_train_steps,
                        num_seed_steps=num_seed_steps,
                        eval_frequency=eval_frequency,
                        num_eval_episodes=num_eval_episodes,
                        replay_buffer=replay_buffer)

            agent.save(f'{args.task}_final.pth')
        else:
            print('loading pretrained', args.task)
            agent.load(f'{args.task}_final.pth')

        # final evaluation

In [ ]:
if args.task == "pg":
    evaluate(env, policy, num_validation_runs=100, render=args.render)
else:
    evaluate_agent(env, agent, "final", num_episodes=100, verbose=True)

In [ ]:
from rollouts import rollout_frames
if args.env == 'pendulum':
    env = gym.make("InvertedPendulum-v5", render_mode='rgb_array')
    max_episode_steps = 200
    env = gym.wrappers.TimeLimit(env, max_episode_steps=max_episode_steps)
elif args.env == 'ant':
    max_episode_steps = 500
    env = gym.make("Ant-v5", render_mode='rgb_array')
    env = gym.wrappers.TimeLimit(env, max_episode_steps=max_episode_steps)

frames = rollout_frames(env, agent)
# show the video, from the list of rgb frames

# Assume 'frames' is a list of images (numpy arrays)
# Example: frames = [frame1, frame2, frame3, ...]

# Define the video file name
video_name = 'output_video.mp4'

# Create a video writer object using imageio
with imageio.get_writer(video_name, fps=25) as writer:
    for frame in frames:
        # Convert the frame to an image and add it to the video
        writer.append_data(frame)

# Display the video in the notebook
Video(video_name)